In [1]:
import numpy as np
import glob
from scipy.optimize import root_scalar

In [2]:
cfs = glob.glob('complete/*.txt')

In [3]:
for file in cfs:
    with open(file, 'r') as in_file:
        noz_length = float(in_file.readlines()[1].replace('#', '').strip())
    ar = np.genfromtxt(file, delimiter=',', skip_header=2)
    throat = ar[0, 0]
    reduced_ar = ar[ar[:, 0] >= throat, 0:2]
    reduced_ar = reduced_ar[reduced_ar[:, 0] <= 1.5 * throat]
    reduced_ar[:, 0] -= throat
    _, validIdx = np.unique(reduced_ar[:, 0], return_index=True)
    reduced_ar = reduced_ar[validIdx, :]
    reduced_ar = np.hstack([reduced_ar, np.zeros((reduced_ar.shape[0], 1))])
    
    rc = 6

    r = rc * reduced_ar[0, 1]
    y0 = reduced_ar[0, 1]

    x1 = reduced_ar[1, 0]
    y1 = reduced_ar[1, 1]

    def obj_fun(theta):
        yt = y0 * (rc+1) - r * np.cos(theta)
        xt = r * np.sin(theta)

        return (y1 - yt) / (x1 - xt) - np.tan(theta)

    theta_end = root_scalar(obj_fun, method='bisect', x0=np.pi/6, bracket=[0, np.pi/2-.01]).root
    theta_start = -np.pi/3

    theta_s = np.linspace(theta_start, theta_end, 15).reshape(-1, 1)
    
    circ_xyz = np.hstack([
        r * np.sin(theta_s),
        y0 * (rc + 1) - r * np.cos(theta_s),
        np.zeros(theta_s.shape)
    ])
    
    corrected_ar = np.vstack([circ_xyz, reduced_ar[1:, :]])
    
    
    fname = file.split('\\')[1]
    np.savetxt(f'SolidWorks/{fname}', corrected_ar, delimiter=' ')